## Pymatgen decomposition energy
Pymatgen has code to compute the phase diagram for a composition, and use it to compute the decomposition energy
- https://pymatgen.org/pymatgen.analysis.phase_diagram.html
  - https://pymatgen.org/pymatgen.analysis.phase_diagram.html#pymatgen.analysis.phase_diagram.PhaseDiagram.get_decomp_and_e_above_hull

See how it compares to the decomposition energy from our scripts

In [1]:
import os
import sys
import numpy as np
# import psycopg2
import sqlalchemy
import pandas as pd
from collections import defaultdict
from tqdm.notebook import tqdm
tqdm.pandas()

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(
    context='talk',
    # context='paper',
    # font_scale=8/8.8,
#     context="talk",
    style='ticks',
    color_codes=True,
    rc={'legend.frameon': False})

plt.rcParams['svg.fonttype'] = 'none'

%matplotlib inline

In [2]:
print(np.__version__)
print(pd.__version__)

1.22.1
1.3.5


In [3]:
import nfp
print(nfp.__version__)

2022-05-25 23:14:34.245973: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /nopt/slurm/current/lib:
2022-05-25 23:14:34.246002: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


0.3.6


In [4]:
os.chdir('../')

In [5]:
print(os.getcwd())
sys.path.append('../../')
import rlmolecule
from rlmolecule.sql.run_config import RunConfig
from rlmolecule.sql import Base, Session
from rlmolecule.sql.tables import GameStore, RewardStore, StateStore
from scripts import ehull

/home/jlaw/projects/arpa-e/crystals/rlmolecule/examples/crystal_energy


In [6]:
# Dataframe containing competing phases from NRELMatDB
print("Reading inputs/competing_phases.csv")
df_competing_phases = pd.read_csv('inputs/competing_phases.csv')
print(f"\t{len(df_competing_phases)} lines")
print(df_competing_phases.head(2))

Reading inputs/competing_phases.csv
	12682 lines
  sortedformula   icsdnum  energyperatom reduced_composition
0    Ag10Br3Te4  173116.0      -1.718985          Ag10Br3Te4
1   Ag11K1O16V4  391344.0      -4.797702         Ag11K1O16V4


In [16]:
# convert the competing phases into pymatgen PD Entries
# https://pymatgen.org/pymatgen.analysis.phase_diagram.html#pymatgen.analysis.phase_diagram.PDEntry
from pymatgen.analysis.phase_diagram import PhaseDiagram, PDEntry
from pymatgen.core.composition import Composition
from pymatgen.core.periodic_table import Element
composition = Composition("Ag10Br3Te4")
pd_entry = PDEntry(composition, -1.718985)
pd_entry

PDEntry : Ag10 Br3 Te4 with energy = -1.7190

In [26]:
# ---FERE reference chemical potentials-----
ferev2_chempot = {'Ag': -0.79, 'Al': -3.27, 'Al_anion': -3.55, 'As': -4.95, 'As_cation': -4.42, 'Au': -1.96, 'B': -6.73,
                  'B_anion': -6.44, 'Ba': -1.44, 'Be': -3.50, 'Bi_cation': -4.22, 'Bi': -4.19, 'Br': -1.54, 'C': -8.94,
                  'Ca': -1.78, 'Cd': -0.64, 'Cl': -1.74, 'Co': -4.67, 'Cr': -7.08, 'Cu': -1.87, 'F': -1.44, 'Fe': -6.00,
                  'Ga': -2.53, 'Ge': -4.34, 'Ge_anion': -4.84, 'Hf': -7.38, 'Hg': -0.10, 'I': -1.53, 'In': -2.39,
                  'Ir': -6.31, 'K': -0.79, 'La': -3.76, 'Li': -1.58, 'Mg': -1.23, 'Mo': -7.37, 'Mn': -6.86, 'N': -8.46,
                  'Na': -1.02, 'Nb': -6.92, 'Ni': -3.65, 'O': -4.80, 'P': -5.17, 'P_cation': -5.14, 'Pb': -3.85,
                  'Pb_anion': -4.29, 'Pd': -3.00, 'Pt': -3.88, 'Rb': -0.58, 'Rh': -4.66, 'Ru': -6.14, 'S': -4.01,
                  'Sb_cation': -4.13, 'Sb': -4.16, 'Sc': -4.42, 'Se': -3.54, 'Si': -5.30, 'Si_anion': -5.40,
                  'Sn': -3.87, 'Sn_anion': -3.71, 'Sr': -1.32, 'Ta': -8.82, 'Te': -3.18, 'Te_cation': -2.75,
                  'Ti': -5.39, 'V': -6.40, 'W': -9.61, 'Y': -4.96, 'Zn': -0.94, 'Zr': -6.39}
fere_entries = [PDEntry(e, energy) for e, energy in ferev2_chempot.items() if "ion" not in e]

In [35]:
pd_entries = df_competing_phases.progress_apply(
    lambda row: PDEntry(Composition(row.reduced_composition),
                        row.energyperatom), 
    axis=1
)

  0%|          | 0/12682 [00:00<?, ?it/s]

In [36]:
pd_entries = pd.concat([pd.Series(fere_entries), pd_entries]).reset_index()[0]
pd_entries

0                PDEntry : Ag1 with energy = -0.7900
1                PDEntry : Al1 with energy = -3.2700
2                PDEntry : As1 with energy = -4.9500
3                PDEntry : Au1 with energy = -1.9600
4                 PDEntry : B1 with energy = -6.7300
                            ...                     
12735    PDEntry : Sn2 Sr1 Zn2 with energy = -2.8068
12736        PDEntry : Sn2 Ta1 with energy = -5.7608
12737        PDEntry : Sn2 Zr1 with energy = -5.2845
12738    PDEntry : Sn3 Zn1 Zr5 with energy = -5.6885
12739         PDEntry : Li2 O1 with energy = -4.7670
Name: 0, Length: 12740, dtype: object

In [42]:
set(pd_entries[0].composition.elements)

{Element Ag}

In [23]:
import itertools

In [114]:
#id, energyperatom, decomp_energy
#Na3Al1Br6_sg163_icsd_062035_1,-3.128,-0.5049999999999999,
test_comp = Composition("Na3Al1Br6")
test_energy = -3.128
test_entry = PDEntry(Composition(test_comp), test_energy)
elements = set(test_comp.elements)
curr_entries = [e for e in pd_entries \
                if len(set(e.composition.elements) - elements) == 0]
curr_entries += [test_entry]
print(f"{len(curr_entries)} entries match an element")
phase_diagram = PhaseDiagram(curr_entries, elements=elements)
# phase_diagram = PhaseDiagram(pd_entries)
phase_diagram

5 entries match an element


Br-Na-Al phase diagram
4 stable phases: 
Br, Al, NaBr, Na

In [115]:
phase_diagram.get_decomp_and_e_above_hull(test_entry, allow_negative=True)

({PDEntry : Br1 Na1 with energy = -3.0572: 0.6,
  PDEntry : Al1 with energy = -3.2700: 0.1,
  PDEntry : Br1 with energy = -1.5400: 0.30000000000000004},
 1.3933731999999999)

In [116]:
decomp, energy = phase_diagram.get_decomp_and_hull_energy_per_atom(test_comp)
decomp, energy

({PDEntry : Br1 Na1 with energy = -3.0572: 0.6,
  PDEntry : Al1 with energy = -3.2700: 0.1,
  PDEntry : Br1 with energy = -1.5400: 0.30000000000000004},
 -1.7061731999999998)

In [117]:
phase_diagram.get_hull_energy(test_comp)

-17.061732

In [83]:
-17/20

-0.85

In [183]:
phase_diagram.get_decomp_and_phase_separation_energy(test_entry)

({PDEntry : Br1 Na1 with energy = -3.0572: 0.6,
  PDEntry : Al1 with energy = -3.2700: 0.1,
  PDEntry : Br1 with energy = -1.5400: 0.30000000000000004},
 1.3933731999999999)

In [7]:
df = pd.read_csv("/projects/rlmolecule/jlaw/crystal_outputs/2022-05-04/batt-icsd-volpred-KLiNa-no-halides-ScY/viz/pred_vs_dft_corrected.csv", index_col=0)
df.head(2)

,decoration,numatoms,relaxedvolume,energyperatom,comp,id,decomp_energy,pred_energyperatom,pred_decomp_energy
0,K1Sb1F6/K1Sb1F6_sg148_icsd_015343_1,8,132.81,-4.636,K1Sb1F6,K1Sb1F6_sg148_icsd_015343_1,0.00926,-4.608471,0.036789
1,Na1Hf1Hg1I7/Na1Hf1Hg1I7_sg9_icsd_060952_2,40,2016.09,-2.820,Na1Hf1Hg1I7,Na1Hf1Hg1I7_sg9_icsd_060952_2,0.04500,-2.620724,0.245000
